<a href="https://colab.research.google.com/github/samitha278/gpt2-lite/blob/main/repro_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from transformers import GPT2LMHeadModel , pipeline , set_seed

In [4]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
params_dict = model.state_dict()

for k,v in params_dict.items():
  print(k,v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [7]:
set_seed(220064)
gen = pipeline('text-generation' , model = 'gpt2')
prompt = "Who is the first president is sri lanka?"
gen(prompt)

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Who is the first president is sri lanka? sri sri lanka?\n\nPresident\n\npresident, sri, president, sri, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, president, presid